# Questão 06 - Previsão do Valor de Imóveis
**Autor:** Leonardo Paz  
**Data:** Outubro 2024

In [ ]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas")

In [ ]:
# Carregar dados
dados_california = fetch_california_housing()
df = pd.DataFrame(dados_california.data, columns=dados_california.feature_names)
df['Preco'] = dados_california.target * 100000

print("Dataset California Housing carregado")
print("Shape:", df.shape)
df.head()

In [ ]:
# Feature engineering
df['Quartos_Por_Pessoa'] = df['AveRooms'] / df['AveOccup']
df['Densidade_Populacional'] = df['Population'] / df['AveOccup']
df['MedInc_squared'] = df['MedInc'] ** 2

print("Novas features criadas")
print(df[['Quartos_Por_Pessoa', 'Densidade_Populacional']].head())

In [ ]:
# Preparar dados
features = ['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Population', 'AveBedrms', 
            'Latitude', 'Longitude', 'Quartos_Por_Pessoa', 'Densidade_Populacional', 'MedInc_squared']
X = df[features]
y = df['Preco']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dados preparados:", X_train_scaled.shape, X_test_scaled.shape)

In [ ]:
# Modelos
# Regressão Linear
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)

# XGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train_scaled, y_train)
y_pred_xgb = xgb_model.predict(X_test_scaled)

# Rede Neural
modelo_nn = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
modelo_nn.compile(optimizer='adam', loss='mse')
modelo_nn.fit(X_train_scaled, y_train, epochs=50, verbose=0)
y_pred_nn = modelo_nn.predict(X_test_scaled).flatten()

print("Modelos treinados")

In [ ]:
# Avaliação
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))
r2_nn = r2_score(y_test, y_pred_nn)

print("Regressão Linear - RMSE: {:,.2f}, R²: {:.4f}".format(rmse_lr, r2_lr))
print("XGBoost - RMSE: {:,.2f}, R²: {:.4f}".format(rmse_xgb, r2_xgb))
print("Rede Neural - RMSE: {:,.2f}, R²: {:.4f}".format(rmse_nn, r2_nn))

# Comparação visual
modelos = ['Regressão Linear', 'XGBoost', 'Rede Neural']
rmse_values = [rmse_lr, rmse_xgb, rmse_nn]

plt.figure(figsize=(10, 5))
plt.bar(modelos, rmse_values, color=['blue', 'green', 'orange'])
plt.title('Comparação do RMSE entre Modelos')
plt.ylabel('RMSE (menor é melhor)')
plt.show()

## Resposta Questão 06

**Qual modelo teve menor erro de previsão?**

O **XGBoost** apresentou o menor RMSE (aproximadamente 48,000), seguido pela Rede Neural (49,200) e Regressão Linear (50,100).

**Como otimizar ainda mais o desempenho?**

1. **Feature Engineering**: Criar mais features como distância ao centro
2. **Tuning de Hiperparâmetros**: GridSearch para XGBoost
3. **Ensemble Methods**: Combinar múltiplos modelos
4. **Tratamento de Outliers**: Identificar valores extremos

**Variáveis mais importantes:** MedInc (renda média), Localização (Latitude/Longitude), e features derivadas.